## 변수 위치 인자를 사용해 시각적인 잡음을 줄여라

위치 인자(positional argument)를 가변적으로 받을 수 있으면 함수 호출이 더 깔끔해지고 시각적 잡음도 줄어든다.

예를 들어 디버깅 정보를 로그에 남기고 싶다고 하자. 인자 수가 고정돼 있으면 메시지와 값의 List를 받는 함수가 필요하다.

In [3]:
def log(message, values):
    if not values:
        print(message)
    else:
        value_str = ', '.join(str(x) for x in values)
        print(f'{message}: {value_str}')

log('내 숫자는', [1, 2])
log('안녕', [])

내 숫자는: 1, 2
안녕


로그에 남길 값이 없을 때도 빈 리스트를 넘겨야 한다면 코드 잡음이 존재한다. 이럴 때 두 번째 인자를 완전히 생략하면 좋다.

파이썬에서는 마지막 위치 인자 이름 앞에 `*`를 붙이면 된다. 로그 메시지의 첫 번째 파라미터는 반드시 필요하지만, 그 이후의 모든 위치 인자는 선택 사항이다.

가변 인자를 써도 함수 본문은 바뀌지 않는다. 단지 호출하는 코드만 바뀐다.

In [4]:
def log(message, *values):
    if not values:
        print(message)
    else:
        value_str = ', '.join(str(x) for x in values)
        print(f'{message}: {value_str}')

log('내 숫자는', [1, 2])
log('안녕')

내 숫자는: [1, 2]
안녕


이미 시퀀스가 있는데 log와 같은 가변 인자 함수에 시퀀스를 사용하고 싶다면 `*` 연산자를 사용하면 된다.

`*` 연산자는 파이썬이 시퀀스의 원소들을 함수의 위치 인자로 넘길 것을 명령한다.

In [5]:
favorites = [7, 33, 99]

log('좋아하는 숫자는', *favorites)

좋아하는 숫자는: 7, 33, 99


가변적인 위치 인자를 받는 데는 두 가지 문제점이 있다.

첫 번째 문제점은 이런 선택적인 위치 인자가 함수에 전달되기 전에 항상 튜플로 변환된다는 것이다. 이는 함수를 호출하는 쪽에서 제너레이터 앞에 `*` 연산자를 사용하면
제너레이터의 모든 원소를 얻기 위해 반복한다는 뜻이다.

이렇게 만들어지는 튜플은 제너레이터가 만들어낸 모든 값을 포함하며, 이로 인해 메모리를 아주 많이 소비하거나 프로그램이 중단돼버릴 수 있다.

In [13]:
def my_generator():
    for i in range(10):
        yield i
        
def my_func(*args):
    print(args)
    
it = my_generator()
my_func(*it)

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)


`*args` 를 받는 함수는 인자 목록에서 가변적인 부분에 들어가는 인자의 개수가 처리하기 좋을 정도로 충분히 작다는 사실을 이미 알고 있는 경우에 가장 적합하다.
- `*args` 는 여러 리터럴이나 변수 이름을 함께 전달하는 함수 호출에 이상적이다.
- `*args` 는 주로 프로그래머의 편의와 코드 가독성을 위한 기능이다.

`*args` 의 두 번째 문제점은 함수에 새로운 위치 인자를 추가하면 해당 함수를 호출하는 모든 코드를 변경해야 한다는 것이다.
- 가변 인자가 존재하는 함수 인자 목록의 앞부분에 위치 인자를 추가하려고 시도하면, 기존 호출 코드를 변경하지 않는 경우 호출하는 코드가 미묘하게 깨질 수 있다.

In [14]:
def log(sequence, message, *values):
    if not values:
        print(f'{sequence} - {message}')
    else:
        value_str = ', '.join(str(x) for x in values)
        print(f'{sequence} - {message}: {value_str}')
#         print(f'{message}: {value_str}')

log(1, '좋아하는 숫자는', 7, 33)
log(1, '안녕')
log('좋아하는 숫자는', 7, 33)

1 - 좋아하는 숫자는: 7, 33
1 - 안녕
좋아하는 숫자는 - 7: 33


세 번째 log 호출에서는 sequence가 주어지지 않았기 때문에 7일 message 파라미터로 사용한다는 점이 문제가 된다.

예외가 발생하지 않고 코드가 작동할 수도 있기 때문에 이런 버그는 추적하기가 어렵다.

이런 가능성을 완전히 없애려면 `*args` 를 받아들이는 함수를 확장할 때는 키워드 기반의 인자만 사용해야 한다.

더 방어적으로 프로그래밍하려면 타입 애너테이션을 사용해도 된다.